In [1]:
from fastai.vision.all import*

In [2]:
import torch
from torch import nn, optim
from torch.functional import F
from torch.utils.data import DataLoader, random_split
from torchvision import transforms, datasets
import torchvision.models as models

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm  
from sklearn.decomposition import PCA
import umap
import math
import io
import umap.plot
import plotly.graph_objs as go 
import plotly.io as pio 
pio.renderers.default ='iframe'


from collections import defaultdict
from tqdm import tqdm
import wandb
import random
from PIL import Image  # Add this import statement

%matplotlib inline
import math,os,sys
import warnings 
warnings.filterwarnings('ignore')

In [3]:
from Network import Network,Network_t ,ContrastiveLoss_with_margin, CustomVGG
from utils import init_weights, init_weights_for_gelu,initialize_weights_mod,plot_activation_stats

In [4]:
import wandb
from fastai.callback.wandb import *
from fastai.data.core import DataLoaders

In [5]:
device= "cpu"
if torch.cuda.is_available():
    device = "cuda"
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device= "mps"

#device= "cpu" #overide device for overfitting a very small data batch
#net = net.to(device)

device

'mps'

In [6]:
from simple_cnn_dataset import TransformedDataset 
from mnist_dataset import MNISTDataset 
#load dataset 
data = pd.read_csv('../data/train.csv')
#data_shorted = data[:10000]
data_clp = data[:1000].reset_index(drop=True) # the index has to be readjusted otherwise it follows the whole data index... problematic
val_count = 2000
#temporarly trying to overfit with less data


#common transformation
default_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(0.5,0.5)
])

In [13]:
train_dataset = MNISTDataset(data.iloc[:-val_count], default_transform)
val_dataset = MNISTDataset(data.iloc[-val_count:], default_transform)

def contrastive_collate(batch):
    first = torch.stack([item[0] for item in batch])
    second = torch.stack([item[1] for item in batch])
    dis = torch.tensor([item[2] for item in batch])
    label = torch.tensor([item[3] for item in batch])
    return (first, second), dis
    
# Define batch size and other DataLoader parameters
batch_size = 128 
num_workers = 2

# Create DataLoaders
dls = DataLoaders.from_dsets(train_dataset, val_dataset, bs=batch_size,collate_fn=contrastive_collate, num_workers=num_workers)

100%|██████████████████████████████████████████████████████████████████| 2000/2000 [00:00<00:00, 3990.94it/s]


In [8]:
from fastai.learner import Learner
from fastai.callback.all import Callback
from fastai.learner import CancelStepException

class ContrastiveLossCallback(Callback):
    def __init__(self, loss_func):
        self.loss_func = loss_func

    def after_pred(self):
        embeddings1 = self.learn.pred[0]  # Assuming embeddings from the model
        embeddings2 = self.learn.pred[1]  # Assuming embeddings from the model
        distance = self.learn.y  # Assuming distance labels (0 or 1)

        loss = self.loss_func(embeddings1, embeddings2, distance)
        self.learn.loss = loss

In [9]:
#dls = DataLoaders(train_loader, val_loader)

In [10]:
#wandb.init(project="Contrastive_learning")


In [15]:
from fastai.learner import Learner

class ContrastiveLearner(Learner):
    def pred_batch(self, batch):
        x, _ = batch
        return self.model(*x)

model = CustomVGG()  # Initialize your custom model
loss_func = ContrastiveLoss_with_margin()  # Initialize your contrastive loss function

# Create the learner with your callback
learn = ContrastiveLearner(dls, model, loss_func=loss_func, cbs=ContrastiveLossCallback(loss_func))
            #(dls, model, loss_func=loss_func, cbs=ContrastiveLossCallback(loss_func))
learn.model.to(device)

CustomVGG(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequ

In [11]:
model = CustomVGG()  # Initialize your custom model
loss_func = ContrastiveLoss_with_margin()  # Initialize your contrastive loss function

learner = Learner(dls, model, loss_func=loss_func, cbs=ContrastiveLossCallback(loss_func))
learner.model.to(device)

CustomVGG(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequ

In [16]:
learn.lr_find()

TypeError: CustomVGG.forward() takes 2 positional arguments but 4 were given

In [ ]:

def get_model(pretrained=False):
    return CustomVGG()
model = CustomVGG()
num_classes=10
dls.c = num_classes
#learner = vision_learner(dls=dls, arch= CustomVGG, metrics=error_rate)
# Create the learner directly
# learner = Learner(dls, model, loss_func=nn.CrossEntropyLoss(), metrics=error_rate,cbs=WandbCallback())
# learner.model.to(device)

In [ ]:
#learner.fit(10, lr=0.00005)

In [ ]:
#learner.fine_tune(10)